In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
dname= 'UWaveGestureLibrary'
file_path = f'./datasets/{dname}/{dname}_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


(315, 120) (120,)


In [2]:
file_path = f'./datasets/{dname}/{dname}_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

(315, 320) (320,)


In [3]:
X_train  = X.transpose()

In [4]:
X_test = X_test.transpose()

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)


In [6]:
num_classes = len(np.unique(y_test))

num_classes

8

In [9]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'multiclass',  # or 'multiclass' for multi-class classification
        'metric': 'multi_logloss',  # or 'multi_logloss' for multi-class classification

        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [10]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:46:15,177] A new study created in memory with name: no-name-04dda8a7-65e3-4f01-b6fe-35c6e8b84e19
[I 2024-02-08 17:46:16,442] Trial 0 finished with value: 0.63125 and parameters: {'num_leaves': 199, 'learning_rate': 0.06027856565946783, 'n_estimators': 729}. Best is trial 0 with value: 0.63125.
[I 2024-02-08 17:46:16,905] Trial 1 finished with value: 0.64375 and parameters: {'num_leaves': 93, 'learning_rate': 0.19268603623123576, 'n_estimators': 307}. Best is trial 1 with value: 0.64375.
[I 2024-02-08 17:46:18,434] Trial 2 finished with value: 0.5125 and parameters: {'num_leaves': 167, 'learning_rate': 0.001805712832684943, 'n_estimators': 472}. Best is trial 1 with value: 0.64375.
[I 2024-02-08 17:46:22,089] Trial 3 finished with value: 0.63125 and parameters: {'num_leaves': 167, 'learning_rate': 0.009710004103341044, 'n_estimators': 973}. Best is trial 1 with value: 0.64375.
[I 2024-02-08 17:46:22,379] Trial 4 finished with value: 0.38125 and parameters: {'num_leaves'

Number of finished trials: 40
Best trial: {'num_leaves': 109, 'learning_rate': 0.13190839257935286, 'n_estimators': 399}
